In [194]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale

In [195]:
df = pd.read_csv('BostonHousing2.csv')
df = df.dropna()

In [196]:
X = df.drop('medv', axis=1)  # Все признаки кроме целевой (цена на недвижку)
lin_reg = LinearRegression()
lin_reg.fit(X_scaled, df['medv'])
print(f"Смещение по оси Y = {lin_reg.intercept_}")
for feature, coef in zip(X.columns, lin_reg.coef_):
    print(f"Коэффициент при {feature}: {coef}")

Смещение по оси Y = 22.561277445109784
Коэффициент при crim: -0.9369479588496202
Коэффициент при zn: 1.0638795439746898
Коэффициент при indus: 0.0960199812896175
Коэффициент при chas: 0.6799342485066063
Коэффициент при nox: -2.099093378994031
Коэффициент при rm: 2.665259399074607
Коэффициент при age: 0.0554827326824389
Коэффициент при dis: -3.1126566507044324
Коэффициент при rad: 2.6691311762884666
Коэффициент при tax: -2.0639978316308007
Коэффициент при ptratio: -2.09090708742805
Коэффициент при b: 0.8536178307970478
Коэффициент при lstat: -3.7467648461339156


Ниже пробую найти все коэфициенты уравнения вручную через градиентный спуск

In [197]:
# загружаем данные, очищаем от пропусков
df = pd.read_csv('BostonHousing2.csv')
df = df.dropna()

In [198]:
# выделяем из столбцов датафрейма признаки и целевую переменную
# (medv - медианная цена домов Y, а все остальное - это иксы X)
X = df.drop('medv', axis=1).values
y = df['medv'].values

Для работы с градиентным спуском масштабируем признаки (вроде должно ускорить процесс нахождения точки минимума).
Как я понял, X_scaled это такая же матрица признаков, просто все значения сведены к одному порядку, чтобы коэфициенты были сопоставимы.

In [199]:
X_mean = X.mean(axis=0) # среднее отклонение по каждому признаку
X_std = X.std(axis=0) # стандартное отклонение по каждому признаку
X_scaled = (X - X_mean) / X_std

In [200]:
X_b = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled] # добавляем столбец для свободного члена (смещение по Y)

In [201]:
# настраиваем параметры градиентного спуска
n_samples, n_features = X_b.shape # число строк и столбцов
theta = np.zeros(n_features) # инициализируем коэффициенты нулями
alpha = 0.1  # скорость обучения
n_iter = 1000 # количество итераций

In [ ]:
# выполняем градиентный спуск
for i in range(n_iter):
    y_pred = X_b @ theta # получаем наше предсказание перемножением матриц

    # (y_pred - y) - вектор ошибок
    # gradient это по сути частная производная функции по каждому из коэфициентов
    # X_b.T - транспонированная матрица признаков (меняем строки и столбцы местами). Если этого не делать выкидыввает ошибку (не сходится размерность матриц)
    gradient = (2 / n_samples) * X_b.T @ (y_pred - y) # по идее если значение градиента стремится к нулю, то мы в точке минимума функции и можем остановиться
    # наверно надо сделать какое-то условие остановки
    theta -= alpha * gradient # обновляем коэфициенты и движемся в сторону минимума функции

In [203]:
for feature, coef in zip(['intercept'] + list(df.drop('medv', axis=1).columns), theta):
    print(f"Коэффициент при {feature}: {coef}")


Коэффициент при intercept: 22.56127744510978
Коэффициент при crim: -0.9369475211502927
Коэффициент при zn: 1.0638787869181892
Коэффициент при indus: 0.09601759637147739
Коэффициент при chas: 0.679934591002348
Коэффициент при nox: -2.099092994101317
Коэффициент при rm: 2.665259825353438
Коэффициент при age: 0.055482363884627815
Коэффициент при dis: -3.1126568806924384
Коэффициент при rad: 2.6691251027387057
Коэффициент при tax: -2.063990940098634
Коэффициент при ptratio: -2.090906861487952
Коэффициент при b: 0.8536177859581849
Коэффициент при lstat: -3.7467646190482164


In [ ]:
#Вывод: значение более-менее совпадают с результатами sklearn, но не совсем.